In [21]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

# As there are possibilities of different chrome
# browser and we are not sure under which it get
# executed let us use the below syntax
from selenium.webdriver.chrome.options import Options

def google_review_parser():
    options = Options()
    options.add_argument('--headless')

    serv = webdriver.chrome.service.Service('./chromedriver')

    driver = webdriver.Chrome(service=serv, options=options)

    # search
    search = '吾米食肆'

    search = '+'.join(search.split(' '))

    url = f'https://www.google.com/maps/place/{search}/?hl=en'
    driver.get(url)

    # process of parsing reviews
    driver.find_element(by=By.XPATH,
                        value='//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()

    # button
    driver.find_element(by=By.XPATH, 
                        value='//*[@id="searchbox-searchbutton"]').click()
    time.sleep(2)

    # button
    driver.find_element(by=By.CLASS_NAME, 
                        value='DkEaL').click()
    time.sleep(2)

    # get number of reivews
    total_number_of_reviews = driver.find_element(By.XPATH, "//*[@class='fontBodySmall']").text.split(" ")[0]
    total_number_of_reviews = int(total_number_of_reviews.replace('.','')) if '.' in total_number_of_reviews else int(total_number_of_reviews)

    #Find scroll layout
    scrollable_div = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")

    count = 0
    #Scroll as many times as necessary to load all reviews
    for i in range(0,(round(total_number_of_reviews/10 - 1))):
        more = driver.find_elements(by=By.XPATH, 
                        value='//*[@aria-label=" See more "]')
        for m in more:
            m.click()
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                scrollable_div)
        time.sleep(2)
        count+=1
        if 2*count > 300: break


    # extract reviews by BeatifulSoup
    response = BeautifulSoup(driver.page_source, 'html.parser')
    texts = response.find_all('span', class_='wiI7pd')
    rates = response.find_all('div', class_='DU9Pgb')


    def get_review_summary(text_set, rate_set):
        rev_dict = {'Review Text': [],
                   'Review Rate': [],
                   'Review Time': []}
        for text, rate in zip(text_set, rates):
            review_rate = rate.find('span', class_='kvMYJc')["aria-label"].strip()[0]
            review_time = rate.find('span',class_='rsqaWe').text
            review_text = text.text
            rev_dict['Review Rate'].append(review_rate)
            rev_dict['Review Time'].append(review_time)
            rev_dict['Review Text'].append(review_text)
        import pandas as pd    
        return(pd.DataFrame(rev_dict))

    df = get_review_summary(texts, rates)

    from datetime import date

    df['search']=search
    df['insert_date']=date.today()
    
    df.to_csv(f'{search}.csv', index=False)
    return df

google_review_parser()

,Review Text,Review Rate,Review Time,search,insert_date
0,(Translated by Google) Ginger Ginger Roast Por...,5,4 months ago,吾米食肆,2022-10-03
1,"(Translated by Google) Ubereats delivery, 45 m...",5,6 months ago,吾米食肆,2022-10-03
2,(Translated by Google) After walking through t...,5,9 months ago,吾米食肆,2022-10-03
3,(Translated by Google) Ubereats delivery\nI or...,5,11 months ago,吾米食肆,2022-10-03
4,(Translated by Google) Very clean and deliciou...,5,a year ago,吾米食肆,2022-10-03
...,...,...,...,...,...
105,,5,2 years ago,吾米食肆,2022-10-03
106,,5,a year ago,吾米食肆,2022-10-03
107,,4,a year ago,吾米食肆,2022-10-03
108,,4,9 months ago,吾米食肆,2022-10-03
